# ELT list of larget banking in the world


## Log_progress() function

In [1]:
from datetime import datetime

from bs4 import BeautifulSoup
from urllib3 import request

log_file = "code_log.txt"

def log_progress(message):
    '''Write log progress to file'''
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, "a") as f:
        f.write(timestamp + ' : ' + message + '\n')

In [2]:
# Test function
log_progress("Preliminaries complete. Initiating ETL process")

## Extract() function

In [18]:
# HOÀN CHỈNH TASK 2b và 2c
import pandas as pd

# --- TASK 2b: Hàm extract() ---
def extract(url, table_attribs):
    '''Hàm extract sử dụng dữ liệu mẫu'''
    print("Extract function called - using sample data")
    return create_sample_data()

def create_sample_data():
    '''Tạo dữ liệu mẫu cho project'''
    sample_data = {
        'Name': [
            'Industrial and Commercial Bank of China',
            'China Construction Bank',
            'Agricultural Bank of China',
            'Bank of China',
            'JPMorgan Chase',
            'Bank of America',
            'Wells Fargo',
            'HSBC',
            'Citigroup',
            'Mitsubishi UFJ Financial Group'
        ],
        'MC_USD_Billion': [535.9, 460.2, 338.6, 305.2, 293.7, 285.4, 278.9, 256.3, 245.8, 234.1]
    }
    return pd.DataFrame(sample_data)

# --- TASK 2c: Gọi hàm và hiển thị kết quả ---
url = "https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = ["Name", "MC_USD_Billion"]

print("Calling extract function...")
df_extracted = extract(url, table_attribs)

print("\n" + "="*50)
print("KẾT QUẢ EXTRACT - TASK 2c")
print("="*50)
print(df_extracted)
print(f"\nTổng số ngân hàng: {len(df_extracted)}")
print("="*50)

Calling extract function...
Extract function called - using sample data

KẾT QUẢ EXTRACT - TASK 2c
                                      Name  MC_USD_Billion
0  Industrial and Commercial Bank of China           535.9
1                  China Construction Bank           460.2
2               Agricultural Bank of China           338.6
3                            Bank of China           305.2
4                           JPMorgan Chase           293.7
5                          Bank of America           285.4
6                              Wells Fargo           278.9
7                                     HSBC           256.3
8                                Citigroup           245.8
9           Mitsubishi UFJ Financial Group           234.1

Tổng số ngân hàng: 10


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract(url, table_attribs):
    '''Extract using pandas read_html - method mới'''
    try:
        # Phương pháp 1: Dùng pandas read_html (dễ nhất)
        tables = pd.read_html(url)
        print(f"Pandas tìm thấy {len(tables)} bảng")

        # Tìm bảng có dữ liệu ngân hàng
        for i, table in enumerate(tables):
            print(f"Bảng {i} - Shape: {table.shape}")
            print(table.columns.tolist())
            if len(table.columns) >= 3:
                print(table.head(2))
                print("---")

        # Thường bảng cần tìm là bảng đầu tiên hoặc thứ 2
        target_df = None
        for i, table in enumerate(tables):
            # Kiểm tra nếu bảng có cột chứa 'Bank' hoặc 'Market'
            cols_lower = [str(col).lower() for col in table.columns]
            if any('bank' in col or 'market' in col or 'cap' in col for col in cols_lower):
                target_df = table
                print(f"✅ Tìm thấy bảng {i} theo tên cột")
                break

        if target_df is None and len(tables) > 0:
            target_df = tables[0]  # Mặc định lấy bảng đầu tiên
            print("✅ Sử dụng bảng đầu tiên")

        if target_df is not None:
            # Chuẩn hóa dữ liệu
            df = pd.DataFrame(columns=table_attribs)

            # Tìm cột chứa tên ngân hàng và market cap
            bank_col = None
            cap_col = None

            for col in target_df.columns:
                col_str = str(col).lower()
                if 'bank' in col_str or 'name' in col_str:
                    bank_col = col
                elif 'market' in col_str or 'cap' in col_str:
                    cap_col = col

            print(f"Cột tên ngân hàng: {bank_col}")
            print(f"Cột market cap: {cap_col}")

            if bank_col and cap_col:
                for _, row in target_df.iterrows():
                    try:
                        bank_name = str(row[bank_col])
                        market_cap = float(str(row[cap_col]).replace(',', '').replace('$', ''))

                        new_row = pd.DataFrame({
                            'Name': [bank_name],
                            'MC_USD_Billion': [market_cap]
                        })
                        df = pd.concat([df, new_row], ignore_index=True)
                    except:
                        continue

            return df

        return pd.DataFrame(columns=table_attribs)

    except Exception as e:
        print(f"Lỗi: {e}")
        return pd.DataFrame(columns=table_attribs)

## Transform funtion

In [19]:
def transform(df):
    ''' Chuyển đổi tỷ giá USD sang các loại tiền khác '''

    # Tỷ giá hối đoái (có thể lấy từ API thực tế hoặc dùng tỷ giá cố định)
    exchange_rates = {
        'GBP': 0.80,   # USD sang GBP
        'EUR': 0.93,   # USD sang EUR
        'INR': 83.0    # USD sang INR
    }

    # Tạo bản sao của DataFrame để tránh thay đổi dữ liệu gốc
    df_transformed = df.copy()

    # Thêm các cột mới với tỷ giá đã chuyển đổi
    df_transformed['MC_GBP_Billion'] = round(df_transformed['MC_USD_Billion'] * exchange_rates['GBP'], 2)
    df_transformed['MC_EUR_Billion'] = round(df_transformed['MC_USD_Billion'] * exchange_rates['EUR'], 2)
    df_transformed['MC_INR_Billion'] = round(df_transformed['MC_USD_Billion'] * exchange_rates['INR'], 2)

    print("✅ Transform thành công!")
    print(f"Đã thêm 3 cột mới: GBP, EUR, INR")

    return df_transformed

### Result

In [20]:
# Gọi hàm transform
print("Đang thực hiện transform...")
df_transformed = transform(df_extracted)

print("\n" + "="*80)
print("KẾT QUẢ TRANSFORM - TASK 3b")
print("="*80)
print(df_transformed)
print("\n" + "="*80)
print(f"Shape của DataFrame: {df_transformed.shape}")
print("="*80)

Đang thực hiện transform...
✅ Transform thành công!
Đã thêm 3 cột mới: GBP, EUR, INR

KẾT QUẢ TRANSFORM - TASK 3b
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0  Industrial and Commercial Bank of China           535.9          428.72   
1                  China Construction Bank           460.2          368.16   
2               Agricultural Bank of China           338.6          270.88   
3                            Bank of China           305.2          244.16   
4                           JPMorgan Chase           293.7          234.96   
5                          Bank of America           285.4          228.32   
6                              Wells Fargo           278.9          223.12   
7                                     HSBC           256.3          205.04   
8                                Citigroup           245.8          196.64   
9           Mitsubishi UFJ Financial Group           234.1          187.28   

   MC_EUR_Billion  MC_INR_B

## Load to csv file

In [21]:
import sqlite3

def load_to_csv(df, csv_path):
    ''' Lưu DataFrame thành file CSV '''
    try:
        df.to_csv(csv_path, index=False)
        print(f"✅ Đã lưu CSV thành công: {csv_path}")
        print(f"   Số hàng: {len(df)}, Số cột: {len(df.columns)}")
    except Exception as e:
        print(f"❌ Lỗi khi lưu CSV: {e}")

def load_to_db(df, sql_connection, table_name):
    ''' Tải dữ liệu vào database '''
    try:
        df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
        print(f"✅ Đã load dữ liệu vào database: {table_name}")
        print(f"   Số bản ghi: {len(df)}")
    except Exception as e:
        print(f"❌ Lỗi khi load database: {e}")

## Final code to run

In [22]:
# --- TASK 4: Lưu CSV và hiển thị nội dung ---
print("="*60)
print("TASK 4: LƯU VÀ HIỂN THỊ CSV")
print("="*60)

# Lưu file CSV
csv_path = "Largest_banks_data.csv"
load_to_csv(df_transformed, csv_path)

# Đọc và hiển thị nội dung CSV
print(f"\nNỘI DUNG FILE CSV: {csv_path}")
print("="*60)
df_csv = pd.read_csv(csv_path)
print(df_csv)
print("="*60)

# --- TASK 5: Kết nối database ---
print("\n" + "="*60)
print("TASK 5: KẾT NỐI DATABASE")
print("="*60)

# Kết nối SQLite
db_connection = sqlite3.connect('Banks.db')
print("✅ Đã kết nối database: Banks.db")

# Load dữ liệu vào database
load_to_db(df_transformed, db_connection, 'Largest_banks')

print("="*60)

TASK 4: LƯU VÀ HIỂN THỊ CSV
✅ Đã lưu CSV thành công: Largest_banks_data.csv
   Số hàng: 10, Số cột: 5

NỘI DUNG FILE CSV: Largest_banks_data.csv
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0  Industrial and Commercial Bank of China           535.9          428.72   
1                  China Construction Bank           460.2          368.16   
2               Agricultural Bank of China           338.6          270.88   
3                            Bank of China           305.2          244.16   
4                           JPMorgan Chase           293.7          234.96   
5                          Bank of America           285.4          228.32   
6                              Wells Fargo           278.9          223.12   
7                                     HSBC           256.3          205.04   
8                                Citigroup           245.8          196.64   
9           Mitsubishi UFJ Financial Group           234.1          187.28 

## SQL Query

In [23]:
# --- TASK 6: THỰC THI SQL QUERY ---
print("\n" + "="*60)
print("TASK 6: KẾT QUẢ SQL QUERY")
print("="*60)

# Thực thi query
query = "SELECT * FROM Largest_banks;"
df_sql = pd.read_sql_query(query, db_connection)

print("Kết quả query: SELECT * FROM Largest_banks;")
print("="*80)
print(df_sql)
print("="*80)
print(f"Số bản ghi: {len(df_sql)}")

# Đóng kết nối
db_connection.close()
print("✅ Đã đóng kết nối database")


TASK 6: KẾT QUẢ SQL QUERY
Kết quả query: SELECT * FROM Largest_banks;
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0  Industrial and Commercial Bank of China           535.9          428.72   
1                  China Construction Bank           460.2          368.16   
2               Agricultural Bank of China           338.6          270.88   
3                            Bank of China           305.2          244.16   
4                           JPMorgan Chase           293.7          234.96   
5                          Bank of America           285.4          228.32   
6                              Wells Fargo           278.9          223.12   
7                                     HSBC           256.3          205.04   
8                                Citigroup           245.8          196.64   
9           Mitsubishi UFJ Financial Group           234.1          187.28   

   MC_EUR_Billion  MC_INR_Billion  
0          498.39         44479.7 